# Build model V0.0

# Net Data
    
| batch=1 | subdivisions=1 | Training |
| :---: | :---: | :---: |
| decay=0.0005 | angle=0 | saturation = 1.5 |
| batch=64 | subdivisions=16 | width=608 |
| height=608 | channels=3 | momentum=0.9 |
|exposure = 1.5 | hue=.1 | learning_rate=0.001|
|burn_in=1000| max_batches = 500200| policy=steps|  
|steps=400000,450000|scales=.1,.1||
 

In [1]:
import cv2
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Dense, Conv2D, LeakyReLU, ZeroPadding2D
from keras.layers import concatenate
from keras.activations import linear
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adagrad
from keras.utils import plot_model
from keras.engine.topology import Layer
import tensorflow as tf

Using TensorFlow backend.


# Statics

In [2]:
nb_class = 80
max_box_per_image = 20

# Prediction Layer

In [3]:
class YoloLayer(Layer):
    def __init__(self, anchors, max_grid, batch_size, warmup_batches, ignore_thresh, 
                    grid_scale, obj_scale, noobj_scale, xywh_scale, class_scale, 
                    **kwargs):
        # make the model settings persistent
        self.ignore_thresh  = ignore_thresh
        self.warmup_batches = warmup_batches
        self.anchors        = tf.constant(anchors, dtype='float', shape=[1,1,1,3,2])
        self.grid_scale     = grid_scale
        self.obj_scale      = obj_scale
        self.noobj_scale    = noobj_scale
        self.xywh_scale     = xywh_scale
        self.class_scale    = class_scale        

        # make a persistent mesh grid
        max_grid_h, max_grid_w = max_grid

        cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(max_grid_w), [max_grid_h]), (1, max_grid_h, max_grid_w, 1, 1)))
        cell_y = tf.transpose(cell_x, (0,2,1,3,4))
        self.cell_grid = tf.tile(tf.concat([cell_x,cell_y],-1), [batch_size, 1, 1, 3, 1])

        super(YoloLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        super(YoloLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        input_image, y_pred, y_true, true_boxes = x

        # adjust the shape of the y_predict [batch, grid_h, grid_w, 3, 4+1+nb_class]
        y_pred = tf.reshape(y_pred, tf.concat([tf.shape(y_pred)[:3], tf.constant([3, -1])], axis=0))
        
        # initialize the masks
        object_mask     = tf.expand_dims(y_true[..., 4], 4)

        # the variable to keep track of number of batches processed
        batch_seen = tf.Variable(0.)        

        # compute grid factor and net factor
        grid_h      = tf.shape(y_true)[1]
        grid_w      = tf.shape(y_true)[2]
        grid_factor = tf.reshape(tf.cast([grid_w, grid_h], tf.float32), [1,1,1,1,2])

        net_h       = tf.shape(input_image)[1]
        net_w       = tf.shape(input_image)[2]            
        net_factor  = tf.reshape(tf.cast([net_w, net_h], tf.float32), [1,1,1,1,2])
        
        """
        Adjust prediction
        """
        pred_box_xy    = (self.cell_grid[:,:grid_h,:grid_w,:,:] + tf.sigmoid(y_pred[..., :2]))  # sigma(t_xy) + c_xy
        pred_box_wh    = y_pred[..., 2:4]                                                       # t_wh
        pred_box_conf  = tf.expand_dims(tf.sigmoid(y_pred[..., 4]), 4)                          # adjust confidence
        pred_box_class = y_pred[..., 5:]                                                        # adjust class probabilities      

        """
        Adjust ground truth
        """
        true_box_xy    = y_true[..., 0:2] # (sigma(t_xy) + c_xy)
        true_box_wh    = y_true[..., 2:4] # t_wh
        true_box_conf  = tf.expand_dims(y_true[..., 4], 4)
        true_box_class = tf.argmax(y_true[..., 5:], -1)         

        """
        Compare each predicted box to all true boxes
        """        
        # initially, drag all objectness of all boxes to 0
        conf_delta  = pred_box_conf - 0 

        # then, ignore the boxes which have good overlap with some true box
        true_xy = true_boxes[..., 0:2] / grid_factor
        true_wh = true_boxes[..., 2:4] / net_factor
        
        true_wh_half = true_wh / 2.
        true_mins    = true_xy - true_wh_half
        true_maxes   = true_xy + true_wh_half
        
        pred_xy = tf.expand_dims(pred_box_xy / grid_factor, 4)
        pred_wh = tf.expand_dims(tf.exp(pred_box_wh) * self.anchors / net_factor, 4)
        
        pred_wh_half = pred_wh / 2.
        pred_mins    = pred_xy - pred_wh_half
        pred_maxes   = pred_xy + pred_wh_half    

        intersect_mins  = tf.maximum(pred_mins,  true_mins)
        intersect_maxes = tf.minimum(pred_maxes, true_maxes)

        intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
        
        true_areas = true_wh[..., 0] * true_wh[..., 1]
        pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

        union_areas = pred_areas + true_areas - intersect_areas
        iou_scores  = tf.truediv(intersect_areas, union_areas)

        best_ious   = tf.reduce_max(iou_scores, axis=4)        
        conf_delta *= tf.expand_dims(tf.to_float(best_ious < self.ignore_thresh), 4)

        """
        Compute some online statistics
        """            
        true_xy = true_box_xy / grid_factor
        true_wh = tf.exp(true_box_wh) * self.anchors / net_factor

        true_wh_half = true_wh / 2.
        true_mins    = true_xy - true_wh_half
        true_maxes   = true_xy + true_wh_half

        pred_xy = pred_box_xy / grid_factor
        pred_wh = tf.exp(pred_box_wh) * self.anchors / net_factor 
        
        pred_wh_half = pred_wh / 2.
        pred_mins    = pred_xy - pred_wh_half
        pred_maxes   = pred_xy + pred_wh_half      

        intersect_mins  = tf.maximum(pred_mins,  true_mins)
        intersect_maxes = tf.minimum(pred_maxes, true_maxes)
        intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
        intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
        
        true_areas = true_wh[..., 0] * true_wh[..., 1]
        pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

        union_areas = pred_areas + true_areas - intersect_areas
        iou_scores  = tf.truediv(intersect_areas, union_areas)
        iou_scores  = object_mask * tf.expand_dims(iou_scores, 4)
        
        count       = tf.reduce_sum(object_mask)
        count_noobj = tf.reduce_sum(1 - object_mask)
        detect_mask = tf.to_float((pred_box_conf*object_mask) >= 0.5)
        class_mask  = tf.expand_dims(tf.to_float(tf.equal(tf.argmax(pred_box_class, -1), true_box_class)), 4)
        recall50    = tf.reduce_sum(tf.to_float(iou_scores >= 0.5 ) * detect_mask  * class_mask) / (count + 1e-3)
        recall75    = tf.reduce_sum(tf.to_float(iou_scores >= 0.75) * detect_mask  * class_mask) / (count + 1e-3)    
        avg_iou     = tf.reduce_sum(iou_scores) / (count + 1e-3)
        avg_obj     = tf.reduce_sum(pred_box_conf  * object_mask)  / (count + 1e-3)
        avg_noobj   = tf.reduce_sum(pred_box_conf  * (1-object_mask))  / (count_noobj + 1e-3)
        avg_cat     = tf.reduce_sum(object_mask * class_mask) / (count + 1e-3) 

        """
        Warm-up training
        """
        batch_seen = tf.assign_add(batch_seen, 1.)
        
        true_box_xy, true_box_wh, xywh_mask = tf.cond(tf.less(batch_seen, self.warmup_batches+1), 
                              lambda: [true_box_xy + (0.5 + self.cell_grid[:,:grid_h,:grid_w,:,:]) * (1-object_mask), 
                                       true_box_wh + tf.zeros_like(true_box_wh) * (1-object_mask), 
                                       tf.ones_like(object_mask)],
                              lambda: [true_box_xy, 
                                       true_box_wh,
                                       object_mask])

        """
        Compare each true box to all anchor boxes
        """      
        wh_scale = tf.exp(true_box_wh) * self.anchors / net_factor
        wh_scale = tf.expand_dims(2 - wh_scale[..., 0] * wh_scale[..., 1], axis=4) # the smaller the box, the bigger the scale

        xy_delta    = xywh_mask   * (pred_box_xy-true_box_xy) * wh_scale * self.xywh_scale
        wh_delta    = xywh_mask   * (pred_box_wh-true_box_wh) * wh_scale * self.xywh_scale
        conf_delta  = object_mask * (pred_box_conf-true_box_conf) * self.obj_scale + (1-object_mask) * conf_delta * self.noobj_scale
        class_delta = object_mask * \
                      tf.expand_dims(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=true_box_class, logits=pred_box_class), 4) * \
                      self.class_scale

        loss_xy    = tf.reduce_sum(tf.square(xy_delta),       list(range(1,5)))
        loss_wh    = tf.reduce_sum(tf.square(wh_delta),       list(range(1,5)))
        loss_conf  = tf.reduce_sum(tf.square(conf_delta),     list(range(1,5)))
        loss_class = tf.reduce_sum(class_delta,               list(range(1,5)))

        loss = loss_xy + loss_wh + loss_conf + loss_class

        loss = tf.Print(loss, [grid_h, avg_obj], message='avg_obj \t\t', summarize=1000)
        loss = tf.Print(loss, [grid_h, avg_noobj], message='avg_noobj \t\t', summarize=1000)
        loss = tf.Print(loss, [grid_h, avg_iou], message='avg_iou \t\t', summarize=1000)
        loss = tf.Print(loss, [grid_h, avg_cat], message='avg_cat \t\t', summarize=1000)
        loss = tf.Print(loss, [grid_h, recall50], message='recall50 \t', summarize=1000)
        loss = tf.Print(loss, [grid_h, recall75], message='recall75 \t', summarize=1000)   
        loss = tf.Print(loss, [grid_h, count], message='count \t', summarize=1000)     
        loss = tf.Print(loss, [grid_h, tf.reduce_sum(loss_xy), 
                                       tf.reduce_sum(loss_wh), 
                                       tf.reduce_sum(loss_conf), 
                                       tf.reduce_sum(loss_class)],  message='loss xy, wh, conf, class: \t',   summarize=1000)   


        return loss*self.grid_scale

    def compute_output_shape(self, input_shape):
        return [(None, 1)]

# Inputs

In [4]:
input_image = Input(shape=(None, None, 3)) # net_h, net_w, 3
true_boxes  = Input(shape=(1, 1, 1, max_box_per_image, 4))
#                          None, None,len(anchors)//6
true_yolo_1 = Input(shape=(None, None, 3, 4+1+nb_class)) # grid_h, grid_w, nb_anchor, 5+nb_class
true_yolo_2 = Input(shape=(None, None, 3, 4+1+nb_class)) # grid_h, grid_w, nb_anchor, 5+nb_class
true_yolo_3 = Input(shape=(None, None, 3, 4+1+nb_class)) # grid_h, grid_w, nb_anchor, 5+nb_class


In [5]:
# Conv
# Layer 1

lay1_1 = Conv2D(32, 3, strides=(1, 1), padding='same')(input_image)
lay1_2 = BatchNormalization(epsilon=0.001)(lay1_1)
lay1_3 = LeakyReLU(alpha=0.1)(lay1_2)

# Downsample

# Conv
# Layer 2

lay2_1 = ZeroPadding2D(((1,0),(1,0)))(lay1_3)
lay2_2 = Conv2D(64, 3, strides=(2, 2), padding='valid')(lay2_1)
lay2_3 = BatchNormalization(epsilon=0.001)(lay2_2)
lay2_4 = LeakyReLU(alpha=0.1)(lay2_3)

# Conv
# Layer 3

lay3_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay2_4)
lay3_2 = BatchNormalization(epsilon=0.001)(lay3_1)
lay3_3 = LeakyReLU(alpha=0.1)(lay3_2)

# Conv
# Layer 4

lay4_1 = Conv2D(64, 3, strides=(1, 1), padding='same')(lay3_3)
lay4_2 = BatchNormalization(epsilon=0.001)(lay4_1)
lay4_3 = LeakyReLU(alpha=0.1)(lay4_2)

# Shortcut -> Conv
# Layer 5

lay5_0 = concatenate([lay2_4,lay4_3], axis=-1)
lay5_1 = Conv2D(32, 1, strides=(1, 1), padding='same')(lay5_0)
lay5_2 = linear(lay5_1)

# Downsample

# Conv
# Layer 6

lay6_1 = ZeroPadding2D(((1,0),(1,0)))(lay5_2)
lay6_2 = Conv2D(128, 3, strides=(2, 2), padding='valid')(lay6_1)
lay6_3 = BatchNormalization(epsilon=0.001)(lay6_2)
lay6_4 = LeakyReLU(alpha=0.1)(lay6_3)

# Conv
# Layer 7

lay7_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay6_4)
lay7_2 = BatchNormalization(epsilon=0.001)(lay7_1)
lay7_3 = LeakyReLU(alpha=0.1)(lay7_2)

# Conv
# Layer 8

lay8_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay7_3)
lay8_2 = BatchNormalization(epsilon=0.001)(lay8_1)
lay8_3 = LeakyReLU(alpha=0.1)(lay8_2)

# Shortcut -> Conv
# Layer 9
lay9_0 = concatenate([lay6_3,lay8_3], axis=-1)
lay9_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay9_0)
lay9_2 = linear(lay9_1)

# Conv
# Layer 10

lay10_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay9_2)
lay10_2 = BatchNormalization(epsilon=0.001)(lay10_1)
lay10_3 = LeakyReLU(alpha=0.1)(lay10_2)

# Conv
# Layer 11

lay11_1 = Conv2D(128, 3, strides=(1, 1), padding='same')(lay10_3)
lay11_2 = BatchNormalization(epsilon=0.001)(lay11_1)
lay11_3 = LeakyReLU(alpha=0.1)(lay11_2)

# Shortcut -> Conv
# layer 12

lay12_0 = concatenate([lay9_2,lay11_3], axis=-1)
lay12_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay12_0)
lay12_2 = linear(lay9_1)

# Downsample

# Conv 
# Layer 13

lay13_1 = ZeroPadding2D(((1,0),(1,0)))(lay12_2)
lay13_2 = Conv2D(256, 3, strides=(2, 2), padding='valid')(lay13_1)
lay13_3 = BatchNormalization(epsilon=0.001)(lay13_2)
lay13_4 = LeakyReLU(alpha=0.1)(lay13_3)

# Conv
# Layer 14

lay14_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay13_4)
lay14_2 = BatchNormalization(epsilon=0.001)(lay14_1)
lay14_3 = LeakyReLU(alpha=0.1)(lay14_2)

# Conv
# Layer 15

lay15_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay14_3)
lay15_2 = BatchNormalization(epsilon=0.001)(lay15_1)
lay15_3 = LeakyReLU(alpha=0.1)(lay15_2)


# Shortcut -> Conv
# Layer 16

lay16_0 = concatenate([lay13_4,lay15_3], axis=-1)
lay16_1 = Conv2D(64, 1, strides=(1, 1), padding='same')(lay16_0)
lay16_2 = linear(lay16_1)


# Conv
# Layer 17

lay17_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay16_2)
lay17_2 = BatchNormalization(epsilon=0.001)(lay17_1)
lay17_3 = LeakyReLU(alpha=0.1)(lay17_2)

# Conv
# Layer 18

lay18_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay17_3)
lay18_2 = BatchNormalization(epsilon=0.001)(lay18_1)
lay18_3 = LeakyReLU(alpha=0.1)(lay18_2)

# Shortcut -> Conv
# Layer 19

lay19_0 = concatenate([lay16_2,lay18_3], axis=-1)
lay19_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay19_0)
lay19_2 = linear(lay19_1)

# Conv
# Layer 20

lay20_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay19_2)
lay20_2 = BatchNormalization(epsilon=0.001)(lay20_1)
lay20_3 = LeakyReLU(alpha=0.1)(lay20_2)

# Conv
# Layer 21

lay21_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay20_3)
lay21_2 = BatchNormalization(epsilon=0.001)(lay21_1)
lay21_3 = LeakyReLU(alpha=0.1)(lay21_2)

# Shortcut -> Conv
# Layer 22

lay22_0 = concatenate([lay19_2,lay21_3], axis=-1)
lay22_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay22_0)
lay22_2 = linear(lay22_1)

# Conv
# Layer 23

lay23_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay22_2)
lay23_2 = BatchNormalization(epsilon=0.001)(lay23_1)
lay23_3 = LeakyReLU(alpha=0.1)(lay23_2)

# Conv 
# Layer 24 

lay24_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay23_3)
lay24_2 = BatchNormalization(epsilon=0.001)(lay24_1)
lay24_3 = LeakyReLU(alpha=0.1)(lay24_2)


# Shortcut -> Conv
# Layer 25

lay25_0 = concatenate([lay22_2,lay24_3], axis=-1)
lay25_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay25_0)
lay25_2 = linear(lay25_1)


# Conv
# Layer 26

lay26_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay25_2)
lay26_2 = BatchNormalization(epsilon=0.001)(lay26_1)
lay26_3 = LeakyReLU(alpha=0.1)(lay26_2)


# Conv
# Layer 27

lay27_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay26_2)
lay27_2 = BatchNormalization(epsilon=0.001)(lay27_1)
lay27_3 = LeakyReLU(alpha=0.1)(lay27_2)

# Shortcut -> Conv
# Layer 28

lay28_0 = concatenate([lay27_3,lay25_2], axis=-1)
lay28_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay28_0)
lay28_2 = linear(lay28_1)

# Conv
# Layer 29

lay29_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay28_2)
lay29_2 = BatchNormalization(epsilon=0.001)(lay29_1)
lay29_3 = LeakyReLU(alpha=0.1)(lay29_2)

# Conv
# Layer 30

lay30_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay29_3)
lay30_2 = BatchNormalization(epsilon=0.001)(lay30_1)
lay30_3 = LeakyReLU(alpha=0.1)(lay30_2)

# # Shortcut -> Conv
# Layer 31

lay31_0 = concatenate([lay30_3,lay28_2], axis=-1)
lay31_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay31_0)
lay31_2 = linear(lay31_1)


# Conv
# Layer 32

lay32_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay31_2)
lay32_2 = BatchNormalization(epsilon=0.001)(lay32_1)
lay32_3 = LeakyReLU(alpha=0.1)(lay32_2)


# Conv
# Layer 33

lay33_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay32_3)
lay33_2 = BatchNormalization(epsilon=0.001)(lay33_1)
lay33_3 = LeakyReLU(alpha=0.1)(lay33_2)

# # Shortcut -> Conv
# Layer 34

lay34_0 = concatenate([lay33_3,lay31_2], axis=-1)
lay34_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay34_0)
lay34_2 = linear(lay34_1)

# Conv
# Layer 35

lay35_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay34_2)
lay35_2 = BatchNormalization(epsilon=0.001)(lay33_1)
lay35_3 = LeakyReLU(alpha=0.1)(lay33_2)

# Conv
# Layer 36

lay36_1 = Conv2D(256, 3, strides=(1, 1), padding='same')(lay35_3)
lay36_2 = BatchNormalization(epsilon=0.001)(lay36_1)
lay36_3 = LeakyReLU(alpha=0.1)(lay36_2)

# # Shortcut -> Conv
# Layer 37

lay37_0 = concatenate([lay36_3,lay34_2], axis=-1)
lay37_1 = Conv2D(128, 1, strides=(1, 1), padding='same')(lay37_0)
lay37_2 = linear(lay37_1)

# Downsample

# Conv
# Layer 38

lay38_1 = ZeroPadding2D(((1,0),(1,0)))(lay37_2)
lay38_2 = Conv2D(512, 3, strides=(2, 2), padding='valid')(lay38_1)
lay38_3 = BatchNormalization(epsilon=0.001)(lay38_2)
lay38_4 = LeakyReLU(alpha=0.1)(lay38_3)

# Conv
# Layer 39

lay39_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay38_4)
lay39_2 = BatchNormalization(epsilon=0.001)(lay39_1)
lay39_3 = LeakyReLU(alpha=0.1)(lay39_2)

# Conv
# Layer 40

lay40_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay39_3)
lay40_2 = BatchNormalization(epsilon=0.001)(lay40_1)
lay40_3 = LeakyReLU(alpha=0.1)(lay40_2)


# Shortcut -> Conv
# Layer 41

lay41_0 = concatenate([lay40_3,lay38_4], axis=-1)
lay41_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay41_0)
lay41_2 = linear(lay41_1)

# Conv
# Layer 42

lay42_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay41_2)
lay42_2 = BatchNormalization(epsilon=0.001)(lay42_1)
lay42_3 = LeakyReLU(alpha=0.1)(lay42_2)


# Conv
# Layer 43

lay43_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay42_2)
lay43_2 = BatchNormalization(epsilon=0.001)(lay43_1)
lay43_3 = LeakyReLU(alpha=0.1)(lay43_2)

# # Shortcut -> Conv
# Layer 44

lay44_1 = concatenate([lay43_3,lay41_2], axis=-1)
lay44_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay44_1)
lay44_3 = linear(lay44_2)


# Conv
# Layer 45

lay45_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay44_3)
lay45_2 = BatchNormalization(epsilon=0.001)(lay45_1)
lay45_3 = LeakyReLU(alpha=0.1)(lay45_2)

# Conv
# Layer 46

lay46_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay45_3)
lay46_2 = BatchNormalization(epsilon=0.001)(lay46_1)
lay46_3 = LeakyReLU(alpha=0.1)(lay46_2)

# # Shortcut -> Conv
# Layer 47

lay47_1 = concatenate([lay46_3,lay44_3], axis=-1)
lay47_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay47_1)
lay47_3 = linear(lay47_2)


# Conv
# Layer 48

lay48_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay47_3)
lay48_2 = BatchNormalization(epsilon=0.001)(lay48_1)
lay48_3 = LeakyReLU(alpha=0.1)(lay48_2)

# Conv
# Layer 49

lay49_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay48_3)
lay49_2 = BatchNormalization(epsilon=0.001)(lay49_1)
lay49_3 = LeakyReLU(alpha=0.1)(lay49_2)

# # Shortcut -> Conv
# Layer 50

lay50_1 = concatenate([lay49_3,lay47_3], axis=-1)
lay50_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay50_1)
lay50_3 = linear(lay50_2)

# Conv
# Layer 51

lay51_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay50_3)
lay51_2 = BatchNormalization(epsilon=0.001)(lay51_1)
lay51_3 = LeakyReLU(alpha=0.1)(lay51_2)

# Conv
# Layer 52

lay52_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay51_3)
lay52_2 = BatchNormalization(epsilon=0.001)(lay52_1)
lay52_3 = LeakyReLU(alpha=0.1)(lay52_2)

# # Shortcut -> Conv
# Layer 53

lay53_1 = concatenate([lay52_3,lay50_2], axis=-1)
lay53_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay53_1)
lay53_3 = linear(lay53_2)


# Conv
# Layer 54

lay54_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay53_3)
lay54_2 = BatchNormalization(epsilon=0.001)(lay54_1)
lay54_3 = LeakyReLU(alpha=0.1)(lay54_2)

# Conv
# Layer 55

lay55_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay54_3)
lay55_2 = BatchNormalization(epsilon=0.001)(lay55_1)
lay55_3 = LeakyReLU(alpha=0.1)(lay55_2)

# # Shortcut -> Conv
# Layer 56

lay56_1 = concatenate([lay55_3,lay53_3], axis=-1)
lay56_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay56_1)
lay56_3 = linear(lay56_2)


# Conv
# Layer 57

lay57_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay56_3)
lay57_2 = BatchNormalization(epsilon=0.001)(lay57_1)
lay57_3 = LeakyReLU(alpha=0.1)(lay57_2)

# Conv
# Layer 58

lay58_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay57_3)
lay58_2 = BatchNormalization(epsilon=0.001)(lay58_1)
lay58_3 = LeakyReLU(alpha=0.1)(lay58_2)

# # Shortcut -> Conv
# Layer 59

lay59_1 = concatenate([lay58_3,lay56_3], axis=-1)
lay59_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay59_1)
lay59_3 = linear(lay59_2)


# Conv
# Layer 60

lay60_1 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay59_3)
lay60_2 = BatchNormalization(epsilon=0.001)(lay60_1)
lay60_3 = LeakyReLU(alpha=0.1)(lay60_2)

# Conv
# Layer 61

lay61_1 = Conv2D(512, 3, strides=(1, 1), padding='same')(lay60_3)
lay61_2 = BatchNormalization(epsilon=0.001)(lay61_1)
lay61_3 = LeakyReLU(alpha=0.1)(lay61_2)

# # Shortcut -> Conv
# Layer 62

lay62_1 = concatenate([lay61_3,lay59_3], axis=-1)
lay62_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay62_1)
lay62_3 = linear(lay62_2)

# # Downsample

# Conv
# Layer 63

lay63_1 = ZeroPadding2D(((1,0),(1,0)))(lay62_3)
lay63_2 = Conv2D(1024, 3, strides=(2, 2), padding='valid')(lay63_1)
lay63_3 = BatchNormalization(epsilon=0.001)(lay63_2)
lay63_4 = LeakyReLU(alpha=0.1)(lay63_3)

# Conv
# Layer 64

lay64_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay63_4)
lay64_2 = BatchNormalization(epsilon=0.001)(lay64_1)
lay64_3 = LeakyReLU(alpha=0.1)(lay64_2)

# Conv
# Layer 65

lay65_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay64_3)
lay65_2 = BatchNormalization(epsilon=0.001)(lay65_1)
lay65_3 = LeakyReLU(alpha=0.1)(lay65_2)

# Shortcut -> Conv
# Layer 66

lay66_1 = concatenate([lay65_3,lay63_3], axis=-1)
lay66_2 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay66_1)
lay66_3 = linear(lay66_2)

# Conv
# Layer 67

lay67_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay66_3)
lay67_2 = BatchNormalization(epsilon=0.001)(lay67_1)
lay67_3 = LeakyReLU(alpha=0.1)(lay67_2)

# Conv
# Layer 68

lay68_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay67_3)
lay68_2 = BatchNormalization(epsilon=0.001)(lay65_1)
lay68_3 = LeakyReLU(alpha=0.1)(lay65_2)

# # Shortcut -> Conv
# Layer 69

lay69_1 = concatenate([lay68_3,lay66_3], axis=-1)
lay69_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay69_1)
lay69_3 = linear(lay69_2)

# Conv
# Layer 70

lay70_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay69_3)
lay70_2 = BatchNormalization(epsilon=0.001)(lay70_1)
lay70_3 = LeakyReLU(alpha=0.1)(lay70_2)

# Conv
# Layer 71

lay71_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay70_3)
lay71_2 = BatchNormalization(epsilon=0.001)(lay71_1)
lay71_3 = LeakyReLU(alpha=0.1)(lay71_2)

# # Shortcut -> Conv
# Layer 72

lay72_1 = concatenate([lay71_3,lay69_3], axis=-1)
lay72_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay72_1)
lay72_3 = linear(lay72_2)


# Conv
# Layer 73

lay73_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay72_3)
lay73_2 = BatchNormalization(epsilon=0.001)(lay73_1)
lay73_3 = LeakyReLU(alpha=0.1)(lay73_2)


# Conv
# Layer 74

lay74_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay73_3)
lay74_2 = BatchNormalization(epsilon=0.001)(lay74_1)
lay74_3 = LeakyReLU(alpha=0.1)(lay74_2)

# Shortcut -> Conv
# Layer 75

lay75_1 = concatenate([lay74_3,lay72_3], axis=-1)
lay75_2 = Conv2D(256, 1, strides=(1, 1), padding='same')(lay75_1)
lay75_3 = linear(lay75_2)

# ######################

# Conv
# Layer 76

lay76_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay75_3)
lay76_2 = BatchNormalization(epsilon=0.001)(lay76_1)
lay76_3 = LeakyReLU(alpha=0.1)(lay76_2)

# Conv
# Layer 77

lay77_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay76_3)
lay77_2 = BatchNormalization(epsilon=0.001)(lay77_1)
lay77_3 = LeakyReLU(alpha=0.1)(lay77_2)

# Conv
# Layer 78

lay78_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay77_3)
lay78_2 = BatchNormalization(epsilon=0.001)(lay78_1)
lay78_3 = LeakyReLU(alpha=0.1)(lay78_2)

# Conv
# Layer 79

lay79_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay78_3)
lay79_2 = BatchNormalization(epsilon=0.001)(lay79_1)
lay79_3 = LeakyReLU(alpha=0.1)(lay79_2)

# Conv
# Layer 80

lay80_1 = Conv2D(512, 1, strides=(1, 1), padding='same')(lay79_3)
lay80_2 = BatchNormalization(epsilon=0.001)(lay80_1)
lay80_3 = LeakyReLU(alpha=0.1)(lay80_2)

# Conv
# Layer 81

lay81_1 = Conv2D(1024, 3, strides=(1, 1), padding='same')(lay80_3)
lay81_2 = BatchNormalization(epsilon=0.001)(lay81_1)
lay81_3 = LeakyReLU(alpha=0.1)(lay81_2)

# Conv
# Layer 82

lay82_1 = Conv2D(255, 1, strides=(1, 1), padding='same')(lay81_3)
lay82_2 = BatchNormalization(epsilon=0.001)(lay82_1)
lay82_3 = LeakyReLU(alpha=0.1)(lay82_2)

pred_yolo_1 = Conv2D((3*(5+nb_class)), 1, strides=(1, 1), padding='same')(lay82_3)
anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
ignore_thresh = .7

loss_yolo_1 = YoloLayer(anchors[12:], 
                        [1*num for num in max_grid], 
                        batch_size, 
                        warmup_batches, 
                        ignore_thresh, 
                        grid_scales[0],
                        obj_scale,
                        noobj_scale,
                        xywh_scale,
                        class_scale)([input_image, pred_yolo_1, true_yolo_1, true_boxes])
# [yolo] 83
# mask = 6,7,8

# classes=80
# num=9
# jitter=.3

# truth_thresh = 1
# random=1


# [route] 84
# layers = -4

# Conv
# Layer 85

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# [upsample] 86
# stride=2

# [route] 87
# layers = -1, 61



# Conv
# Layer 88

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 89

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=512
# activation=leaky

# Conv
# Layer 90

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 91

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=512
# activation=leaky

# Conv
# Layer 92

# batch_normalize=1
# filters=256
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 93

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=512
# activation=leaky

# Conv
# Layer 94

# size=1
# stride=1
# pad=1
# filters=255
# activation=linear


# [yolo] 95
# mask = 3,4,5
# anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
# classes=80
# num=9
# jitter=.3
# ignore_thresh = .7
# truth_thresh = 1
# random=1



# [route] 96
# layers = -4

# Conv
# Layer 97

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# [upsample] 98
# stride=2

# [route] 99
# layers = -1, 36



# Conv 100
# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 101

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# Conv
# Layer 102

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 103

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# Conv
# Layer 104

# batch_normalize=1
# filters=128
# size=1
# stride=1
# pad=1
# activation=leaky

# Conv
# Layer 105

# batch_normalize=1
# size=3
# stride=1
# pad=1
# filters=256
# activation=leaky

# Conv
# Layer 106

# size=1
# stride=1
# pad=1
# filters=255
# activation=linear


# [yolo] 107
# mask = 0,1,2
# anchors = 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326
# classes=80
# num=9
# jitter=.3
# ignore_thresh = .7
# truth_thresh = 1
# random=1



NameError: name 'max_grid' is not defined

In [ ]:
model = Model(inputs=input_image, outputs=lay82_3)
plot_model(model, to_file='model.png',show_shapes='true')